# Descripción del proyecto

La compañía Sweet Lift Taxi ha recopilado datos históricos sobre pedidos de taxis en los aeropuertos. Para atraer a más conductores durante las horas pico, necesitamos predecir la cantidad de pedidos de taxis para la próxima hora. Construye un modelo para dicha predicción.

La métrica RECM en el conjunto de prueba no debe ser superior a 48.

## Instrucciones del proyecto.

1. Descarga los datos y haz el remuestreo por una hora.
2. Analiza los datos
3. Entrena diferentes modelos con diferentes hiperparámetros. La muestra de prueba debe ser el 10% del conjunto de datos inicial.
4. Prueba los datos usando la muestra de prueba y proporciona una conclusión.

## Descripción de los datos

Los datos se almacenan en el archivo `taxi.csv`. 	
El número de pedidos está en la colum


## Preparaciónna `num_orders`.

In [ ]:
# Se importan las librerias para el procesamiento de datos
import time
import pandas as pd
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
from pandas.plotting import autocorrelation_plot
from IPython.display import display

# Se descargan las librerias para la generacion de graficos
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

# Se descargan las librerias para mechine learning
import catboost
import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, make_scorer

In [ ]:
# Se cargan los datos e imprime su estrutura para validar que sea correcta
data = pd.read_csv('/datasets/taxi.csv', parse_dates=[0], index_col=[0])

display(data.info())

***Los datos se descargaron correctamente y el  indice corresponde a las fechas***

In [ ]:
# Se revisan valores ausentes
display(data.isna().sum())

***No se hallan valores ausentes en la serie***

In [ ]:
# Se ordenan los datos por su indice
data.sort_index(inplace=True)

# Se realiza remuestreo de los datos para poder obtener informacion por hora
data = data.resample('H').mean()

display(data.head())

In [ ]:
# Se muestran los valores maximo y minimo de la columna num_orders
print('Valor minimo de la columna num_orders -->', data['num_orders'].min())
print('Valor maximo de la columna num_orders -->', data['num_orders'].max())

***Los datos estan ordenados de forma ascendente por el indice y estan agrupados para mostrar el numero de pedidos por hora***

In [ ]:
# Se grafican los datos iniciales para observar su comportamiento y validar cambios mas adelante en el proyecto

# Se define el estilo del grafico
plt.style.use('seaborn-dark')

# Se crear la figura y ajustar el tamaño
fig, ax = plt.subplots(figsize=(11, 4))

# Se personaliza el grafico
data.plot(ax=ax, color='dodgerblue', marker='o', linestyle='-', linewidth=2)

# Se añaden los título y etiquetas
plt.title('Número de pedidos por hora', fontsize=16, fontweight='bold')
plt.xlabel('Fecha', fontsize=12)
plt.ylabel('Número de pedidos', fontsize=12)

# Se rotan las etiquetas para mayor claridad y se ajusta el espacio para mostrarlas
plt.xticks(rotation=60, ha='right')  

# Se añaden líneas de cuadrícula y personaliza la opacidad
plt.grid(True, which='both', linestyle='--', linewidth=0.7, alpha=0.7)

# Se ajusta la leyenda
ax.legend(loc='upper left', fontsize=10, title='Series de datos')

# Se muestra el gráfico
plt.tight_layout() 
plt.show()


***Se puede observar que la serie temporal tiene tendencia positiva***



## Análisis

In [ ]:
# Se hallan la tendencia, estacionalidad y ruido de los datos
results_decompose_data = seasonal_decompose(data)

# Se crea una figura para los graficos de la tendencia, estacionalidad y ruido de los datos
plt.figure(figsize=[14,10])

# Se define el estilo del grafico
plt.style.use('seaborn-dark')

# Se crea el primer subgrafico
plt.subplot(311)
results_decompose_data.trend.plot(ax=plt.gca())
plt.title('Tendencia')

# Se crea el segundo subgrafico
plt.subplot(312)
results_decompose_data.seasonal.plot(ax=plt.gca())
plt.title('Estacionalidad')

# Se crea el tercer subgrafico
plt.subplot(313)
results_decompose_data.resid.plot(ax=plt.gca())
plt.title('Ruido')

In [ ]:
# Se valida que los componentes graficados correspondan a la serie
display(results_decompose_data.trend[12:16] + results_decompose_data.seasonal[12:16] + results_decompose_data.resid[12:16])
print('')
display(data[12:16])

# Analisis graficos de tendencia, estacionalidad y ruido

Después de revisar los gráficos de la tendencia, estacionalidad y ruido para la serie de datos, se pueden realizar las siguientes inferencias:

- ***Tendencia***: esta es positiva, ya que el valor den en el eje de la Y o número de pedidos, se incrementa con el pasar del tiempo o aumentar los valores en el eje X.

- ***Estacionalidad***: se observa un patrón repetitivo todo el tiempo, lo que indica que la serie tiene periodos de estacionalidad bien definidos.

- ***Ruido***: no se observa patrones definidos, lo que es positivo para el entrenamiento del modelo predictivo.


In [ ]:
# Se genera el grafico de autocorrelacion para validar los resultados obtenidos en los componenetes analizados anteriormente
plt.figure(figsize=(11, 3))
autocorrelation_plot(data)
plt.xlabel('Rezagos')
plt.ylabel('AutoCorrelacion')

# Analisis del grafico de autocorrelacion

Después de revisar el grafico de autocorrelacion para la serie de datos, se pueden realizar las siguientes inferencias:

- ***Tendencia***: la disminución gradual en el grafico indica una tendencia en la serie temporal. Esto también indica que los tienen una fuerte relación en el tiempo, lo que puede sugerir que se repetirán en el futuro
- ***Estacionalidad***: se observan patrones de oscilación aproximadamente cada 750 horas, lo que es equivalente a  30 días. Esto indica un comportamiento repetitivo en los periodos entes mencionados.
- ***Ruido***: El gráfico de autocorrelación muestra patrones claros y una disminución gradual de los valores de autocorrelación, lo que indica la presencia de tendencia y estacionalidad en la serie. Estos patrones también sugieren que el ruido en la serie es limitado y que la estructura subyacente de la serie (tendencia y estacionalidad) domina sobre el componente aleatorio (ruido), lo que sugiere que el ruido está dentro de los parámetros esperados para este tipo de datos.

En resumen, los datos tiene una estructura adecuada para realizar el entrenamiento del modelo predictivo.

In [ ]:
# Se crean caracteristicas para la serie temporal
def create_features(serie, max_lag, rolling_mean_size):
    """
    La fucionn toma tres parametros para poder generar las caracteristicas a la serie temporal.
    La caracteristicas se definiran por tiempo, rezagos y media movil.
    
    Parametros
        
        serie: esta es la serie que se usara para generar las nuevas columnas con las respectivas caracteristicas.
        max_lag: es el numero maximo de rezagos que se usaran para crear caracteristicas.
        rolling_mean_size: es el valor de la venta que se usara para el calculo del promedio.
    
    Al finalizar la funcion, la serie entregada como parametro, tendra cuatro columnas basadas en periodos de tiempo (mes, semana, dia y dia de la semana)
    n columnas para los rezagos y una columna con la media movil.    
    """
    serie['month'] = serie.index.month
    serie['week'] = serie.index.week
    serie['day'] = serie.index.day
    serie['day_of_week'] = serie.index.dayofweek
    
    for lag in range(1, max_lag + 1, 1):
        serie['lag_{}'.format(lag)] = serie['num_orders'].shift(lag)
        
    serie['rolling_mean'] = serie['num_orders'].shift().rolling(rolling_mean_size).mean()

In [ ]:
# Se llama a la funcion de creacion de caracteristicas y se muestran en pantalla las nuevas columnas
create_features(data, 6, 6)
display(data.head())

## Formación

Para el desarrollo del modelo predictivo se usaran los siguientes algoritmos para poder comparar sus resultados y elegir la mejor alternativa:

- LinearRegression
- RandonForestRegressor
- LightGBM
- CatBoost

In [ ]:
# Se eliminan los valores ausentes
data = data.dropna()

# Se crean los conjuntos de entrenamiento y validacion
train, test = train_test_split(data, shuffle=False, test_size=0.1)

# Se validan las dimensiones de los conjuntos obtenidos
print('Conjunto de entranamiento --> ', train.shape)
print('Conjunto de validacion --> ', test.shape)

In [ ]:
# Se obtienen los conjunto de caracteristicas y el objetivo
features_train = train.drop(['num_orders'], axis=1)
target_train = train['num_orders']

features_test = test.drop(['num_orders'], axis=1)
target_test = test['num_orders']
                              
# Se encalan las caracteristicas
scaler = StandardScaler()

features_train_scaled = scaler.fit_transform(features_train)
features_test_scaled = scaler.transform(features_test)
                              
print('Conjunto de las caracteristicas de entrenamiento escaladas --> ', features_train_scaled.shape)
print('Conjunto de las caracteristicas de validacion escaladas --> ', features_test_scaled.shape)

In [ ]:
# Se  crea un diccionario para guardar los resultados entregados por los modelos y poder compararlos
models_results = {'Model':[],
                 'RMSE':[],
                 '% RMSE': [],
                 'Minutes Train':[],
                 'Minutes Predict':[]
                } 


# Se define la funcion que agrega los tados del entrenamiento al diccionario
def train_results(name_model, train_time):
    """
    La funcion toma dos parametros y los guarda en las claves Model y Minutes Train
    
    Parametros:
        name_model: es el nombre del algoritmo que se usa para entrenar el modelo
        train_time: es el tiempo en minutos que se tarda el entrenamiento del modelo
    """
    models_results['Model'].append(name_model)
    models_results['Minutes Train'].append(train_time)
    
# Se define la funcion que agrega los tados de las predicciones al diccionario
def test_results(rmse, percentage_rmse, prediction_time):
    """
    La funcion toma tres parametros y los guarda en las claves RMSE, % RMSE y Minutes Predict
    
    Parametros:
        RMSE           : es la metrica usada para evaluar el desempeño del modelo
        % RMSE         : es el porcentaje de la metrica RMSE con respecto al rango valores en el objetivo predicciones
        prediction_time: es el tiempo en minutos que se tardan las predicciones del modelo
    """
    
    models_results['RMSE'].append(rmse)
    models_results['% RMSE'].append(percentage_rmse) 
    models_results['Minutes Predict'].append(prediction_time)

# Se valida la correcta creacion del diccionario
display(models_results)

In [ ]:
# Se hallan los valores maximo y minimo del objetivo de entrenamiento para poder hallar el porcentaje de RMSE
value_min_target_test = target_test.min()
value_max_target_test = target_test.max()

print('Valor minimo del objetivo usado para validacion --> ', round(value_min_target_test, 2))
print('Valor maximo del objetivo usado para validacion --> ', round(value_max_target_test, 2))

***Entrenamiento para el modelo LinearRegression***

In [ ]:
# Se inicializan las variables que guardan el tiempo de inicio y finalizacion del entrenamiento
train_start_time = 0
train_end_time = 0
total_train_time = 0

# Se entrena el modelo de LinearRegression
train_start_time = time.time()
model_LinearRegression = LinearRegression()
model_LinearRegression.fit(features_train_scaled, target_train)
train_end_time = time.time()
total_train_time = round((train_end_time - train_start_time) / 60, 2)

# Se guardan los resultados del entrenamiento para compararlos con los demas modelos
train_results('LinearRegression', total_train_time)

display(models_results)

In [ ]:
# Se define la funcion que calcula el RMSE y permite usar esta metrica como parametro 
# para que se busquen los mejores hiperparametros

def rmse(target, prediction):
    """
    La funcion toma dos parametros y devuelve el valor del RMSE las predicciones de un modelo
    
    Parametros:
        target    : son los objetivos usando para validar la exactitud las predicicones 
        prediction: son las predicciones entregadas por el modelo
    """
    return np.sqrt(mean_squared_error(target, prediction))

# Se crea un objeto scorer para poder usarlo en los hiperparametros que usaara el modelo 
rmse_scorer = make_scorer(rmse, greater_is_better=False)

In [ ]:
***Entrenamiento para el modelo RandonForestRegressor***

In [ ]:
# Se inicializan las variables que guardan el tiempo de inicio y finalizacion del entrenamiento
train_start_time = 0
train_end_time = 0
total_train_time = 0

# Se entrena el modelo de RandonForestRegressor
train_start_time = time.time()
model_RandomForestRegressor = RandomForestRegressor(random_state=12345)

# Se definen los espacios de busqueda de los hiperparametros
param_grid = {'n_estimators':[20, 40, 60, 80],
              'max_depth':[3, 6, 9, 12]
             }

# Se configura GridSearchCV los mejores hiperparametros basado en RMSE
grid_search_RandomForestRegressor = GridSearchCV(estimator=model_RandomForestRegressor, 
                           param_grid=param_grid, 
                           scoring=rmse_scorer, 
                           cv=4)

# Se entrena el modelo
grid_search_RandomForestRegressor.fit(features_train_scaled, target_train)

print('Mejores hiperparametros para RandomForestRegressor --> ', grid_search_RandomForestRegressor.best_params_)

# Se finaliza  y totaliza el conteo del tiempo de entrenamiento
train_end_time = time.time()
total_train_time = round((train_end_time - train_start_time) / 60, 2)

# Se guardan los resultados del entrenamiento para compararlos con los demas modelos
train_results('RandomFRegressor', total_train_time)
print('')
display(models_results)

***Entrenamiento para el modelo LightGBM***

In [ ]:
# Se inicializan las variables que guardan el tiempo de inicio y finalizacion del entrenamiento
train_start_time = 0
train_end_time = 0
total_train_time = 0

# Se entrena el modelo de XGBoost
train_start_time = time.time()
model_LightGBM = lgb.LGBMRegressor(random_state=12345)

# Se configuran los hiperparametros a ajustar 
param_grid = {
             'num_leaves':[20, 30, 40],
             'learning_rate':[0.01, 0.1, 0.2],
             'n_estimators':[20, 40, 60, 80],
             'max_depth':[3, 6, 9, 12]
             }

# Se configura GridSearchCV los mejores hiperparametros basado en RMSE
grid_search_LightGBM = GridSearchCV(estimator=model_LightGBM, 
                           param_grid=param_grid, 
                           scoring='neg_root_mean_squared_error', 
                           cv=4,
                           n_jobs=-1)

# Se entrena el modelo
grid_search_LightGBM.fit(features_train_scaled, target_train)

print('Mejores hiperparametros para LightGBM --> ', grid_search_LightGBM.best_params_)

# Se finaliza  y totaliza el conteo del tiempo de entrenamiento
train_end_time = time.time()
total_train_time = round((train_end_time - train_start_time) / 60, 2)

# Se guardan los resultados del entrenamiento para compararlos con los demas modelos
train_results('LGBM', total_train_time)
print('')
display(models_results)

***Entrenamiento para el modelo CatBoost***

In [ ]:
# Se inicializan las variables que guardan el tiempo de inicio y finalizacion del entrenamiento
train_start_time = 0
train_end_time = 0
total_train_time = 0

# Se entrena el modelo de XGBoost
train_start_time = time.time()
model_CatBoost = CatBoostRegressor(random_state=12345, logging_level='Silent')

# Se configuran los hiperparametros a ajustar 
param_grid = {
             'l2_leaf_reg':[1, 3, 5],
             'learning_rate':[0.01, 0.1, 0.2],
             'iterations':[20, 40, 60, 80],
             'depth':[3, 6, 9, 12]
             }

# Se configura GridSearchCV los mejores hiperparametros basado en RMSE
grid_search_CatBoost = GridSearchCV(estimator=model_CatBoost, 
                           param_grid=param_grid, 
                           scoring='neg_root_mean_squared_error', 
                           cv=4,
                           n_jobs=-1)

# Se entrena el modelo
grid_search_CatBoost.fit(features_train_scaled, target_train)

print('Mejores hiperparametros para LightGBM --> ', grid_search_CatBoost.best_params_)

# Se finaliza  y totaliza el conteo del tiempo de entrenamiento
train_end_time = time.time()
total_train_time = round((train_end_time - train_start_time) / 60, 2)

# Se guardan los resultados del entrenamiento para compararlos con los demas modelos
train_results('CatBoost', total_train_time)
print('')
display(models_results)

## Prueba

Se procede a generar las predicciones para cada uno de los modelos entrenados.


***Prediccioens para el modelo LinearRegression***

In [ ]:
# Se inicializan las variables que guardan el tiempo de inicio y finalizacion de las predicciones
test_start_time = 0
test_end_time = 0
total_test_time = 0

# Se realizan las predicciones
test_start_time = time.time()
predictions = model_LinearRegression.predict(features_test_scaled)

# Se finaliza y totaliza el tiempo tardado en realizar las prediciones
test_end_time = time.time()
total_test_time = round((test_end_time - test_start_time) / 60, 4)

# Se halla la metrica RMSE
mse = mean_squared_error(target_test, predictions)
rmse = round(mse ** 0.5, 2)

# Se halla el porcentage de RMSE con respecto al rango de valores en el objetivo
percentage_rmse = 0
percentage_rmse = round((rmse / (value_max_target_test - value_min_target_test) * 100), 2)

# Se guardan los resultados de las predicciones para compararlos con los demas modelos
test_results(rmse, percentage_rmse, total_test_time)
print('')
display(models_results)

# Se borran variables para evitar insuficiencia en la memoria
del model_LinearRegression

***Prediccioens para el modelo RandomForestRegressor***

In [ ]:
# Se inicializan las variables que guardan el tiempo de inicio y finalizacion de las predicciones
test_start_time = 0
test_end_time = 0
total_test_time = 0

# Se realizan las predicciones
test_start_time = time.time()

# Evaluar el modelo con los mejores hiperparámetros en el conjunto de validación
# Obtener el mejor modelo
best_model = grid_search_RandomForestRegressor.best_estimator_  
predictions = best_model.predict(features_test_scaled)      

# Se finaliza y totaliza el tiempo tardado en realizar las prediciones
test_end_time = time.time()
total_test_time = round((test_end_time - test_start_time) / 60, 4)

# Se halla la metrica RMSE
mse = mean_squared_error(target_test, predictions)
rmse = round(mse ** 0.5, 2)

# Se halla el porcentage de RMSE con respecto al rango de valores en el objetivo
percentage_rmse = 0
percentage_rmse = round((rmse / (value_max_target_test - value_min_target_test) * 100), 2)

# Se guardan los resultados de las predicciones para compararlos con los demas modelos
test_results(rmse, percentage_rmse, total_test_time)
print('')
display(models_results)

# Se borran variables para evitar insuficiencia en la memoria
del model_RandomForestRegressor, grid_search_RandomForestRegressor

***Prediccioens para el modelo LightGBM***

In [ ]:
# Se inicializan las variables que guardan el tiempo de inicio y finalizacion de las predicciones
test_start_time = 0
test_end_time = 0
total_test_time = 0

# Se realizan las predicciones
test_start_time = time.time()

# Evaluar el modelo con los mejores hiperparámetros en el conjunto de validación
# Obtener el mejor modelo
best_model = grid_search_LightGBM.best_estimator_  
predictions = best_model.predict(features_test_scaled)      

# Se finaliza y totaliza el tiempo tardado en realizar las prediciones
test_end_time = time.time()
total_test_time = round((test_end_time - test_start_time) / 60, 4)

# Se halla la metrica RMSE
mse = mean_squared_error(target_test, predictions)
rmse = round(mse ** 0.5, 2)

# Se halla el porcentage de RMSE con respecto al rango de valores en el objetivo
percentage_rmse = 0
percentage_rmse = round((rmse / (value_max_target_test - value_min_target_test) * 100), 2)

# Se guardan los resultados de las predicciones para compararlos con los demas modelos
test_results(rmse, percentage_rmse, total_test_time)
print('')
display(models_results)

# Se borran variables para evitar insuficiencia en la memoria
del model_LightGBM, grid_search_LightGBM

In [ ]:
***Prediccioens para el modelo CatBoost***

In [ ]:
# Se inicializan las variables que guardan el tiempo de inicio y finalizacion de las predicciones
test_start_time = 0
test_end_time = 0
total_test_time = 0

# Se realizan las predicciones
test_start_time = time.time()

# Evaluar el modelo con los mejores hiperparámetros en el conjunto de validación
# Obtener el mejor modelo
best_model = grid_search_CatBoost.best_estimator_  
predictions = best_model.predict(features_test_scaled)      

# Se finaliza y totaliza el tiempo tardado en realizar las prediciones
test_end_time = time.time()
total_test_time = round((test_end_time - test_start_time) / 60, 4)

# Se halla la metrica RMSE
mse = mean_squared_error(target_test, predictions)
rmse = round(mse ** 0.5, 2)

# Se halla el porcentage de RMSE con respecto al rango de valores en el objetivo
percentage_rmse = 0
percentage_rmse = round((rmse / (value_max_target_test - value_min_target_test) * 100), 2)

# Se guardan los resultados de las predicciones para compararlos con los demas modelos
test_results(rmse, percentage_rmse, total_test_time)
print('')
display(models_results)

In [ ]:
df_models_results = pd.DataFrame(models_results)

# Se crea una figura para graficas los resultados en los entrenamientos de los modelos
plt.figure(figsize=[12,5])

# Se filtran los valores para el eje X (Primer columna)
x = df_models_results.iloc[:,0]

# Se filtran los valores para el eje Y
y_columns = df_models_results.iloc[:,1:]

# Se define el ancho de las barras del grafico
bar_width = 0.1

# Se halla la longitud de la primer columna para usarla como guia para posicionar las barras en el grafico
index = range(len(x))

# Se iteran los datos de y_column con enumerate, que entrega el indice de la columna y su nombre
for i, column in enumerate(y_columns):
    # Se crea un grafico de barras
    # El primer argumento determina la posicion de la barra para que no se superpongan
    # El segundo argumento toma el valor de la columna para determinar la altura de la barra
    # El tercer argumento especifica el ancho de la barra
    # El cuarto argumento asigna el nombre de etiqueta para la barra
    plt.bar([pos + i * bar_width for pos in index], df_models_results[column], bar_width, label=column)
    
plt.xlabel('Algoritmos')
plt.ylabel('Resultados de validacion')
plt.title('Comparacion de resultados de los modelos entrenados')
plt.grid()

plt.xticks([pos + bar_width * (len(y_columns) / 2) for pos in index], x)

plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=len(y_columns), frameon=False)

plt.tight_layout()

plt.show()

# Conclusiones

El conjunto de datos entregado para analizar no presento valores ausentes, y los valores de la columna num_orders estaban dentro de los rangos que representa a dicha columna.

Al revisar la tendencia, estacionalidad y ruido de los datos, se pudo observar una clara tendencia al alza en el número de pedido a medida que transcurren los días. Referente a la estacionalidad, esta se presenta en un ciclo aproximado de 750 horas, que se pueden interpretar como un evento que ocurre cada mes. Este comportamiento de los datos lleva a estudiar con detalle las fechas en que se repite el patrón para así hacer frente al aumento en la demanda y poder maximizar las oportunidades de generar más viajes. Por último, el ruido encontrado en los datos no presento patrón alguno, lo que se traduce en datos que no tienen relevancia o impacto significativo y esto se puede comprobar con la clara definición en los gráficos de tendencia y estacionalidad.

Pasando al entrenamiento de los modelos, se eligieron los siguientes algoritmos para realizarlo:

- LinearRegression
- RandonForestRegressor
- LightGBM
- CatBoost

Se crearon cuatro modelos, uno para cada uno de los algoritmos mencionados anteriormente. Se generaron los listados de los mejores parámetros a usar por los modelos no lineales. Todos los modelos no lineales se entrenaron con el mismo grupo de hiperparametros para de estos seleccionar los que entregan mejores resultados y, los resultados se basaron en la métrica RMSE.

Por último se procedió a ejecutar las pruebas de las predicciones para cada modelo, estos resultados se obtuvieron con el 10% del conjunto de datos que se entregó para este proyecto. 

Después de analizar las métricas a las que se les realizo seguimiento, se observa que los mejores resultados los entrega el modelo basado en el algoritmo LinearRegression, pues tarda muy poco tiempo en su entrenamiento como en las predicciones y arrojo un RMSE de ***8.91***, siendo este el más bajo de todos los resultados entregados por los modelos. Por lo anterior se sugiere utilizar el modelo de predicción basado en el algoritmo LinearRegression para encontrar los datos que busca la empresa. 

A continuación se puede observar el resumen de los resultados que se obtuvieron con las predicciones generadas por los modelos entrenados.


In [ ]:
display(df_models_results)